Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.

In [148]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import os
from splinter import Browser
import re
import pandas as pd

In [149]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### Get first news headline and summary text

In [150]:
# URL of page to be scraped
url = "https://mars.nasa.gov/news"

In [151]:
# Retrieve page with the requests module
#response = requests.get(url)
browser.visit(url)

In [152]:
html = browser.html


In [153]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html,'html.parser')

In [154]:
#soup_output_file = open(r"soup_output.txt","w")
#soup_output_file.write(soup.prettify())
#soup_output_file.close()

In [155]:
#results are returned as an iterable list
# article_title =  soup.find('div', {'class' : "bottom_gradient"}, {'class': 'content_title'})
# article_title =  soup.find('div', 'a', {'class': 'content_title'})
# print(article_title)
# article_title.text

None


AttributeError: 'NoneType' object has no attribute 'text'

In [156]:
article_titles = []
for div in soup.find_all('div', attrs={'class':'content_title'}):
    article_titles.append(div.find('a'))
article_titles[1].text

"NASA's Mars Rover Drivers Need Your Help"

In [162]:
article_summary = soup.find('div', {'class': 'article_teaser_body'})
article_summary.text

'Using an online tool to label Martian terrain types, you can train an artificial intelligence algorithm that could improve the way engineers guide the Curiosity rover.'

### Get featured image

In [46]:
url_featured_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

In [52]:
browser.visit(url_featured_image)


In [54]:
html_featured_image = browser.html
#html_featured_image
soup_featured_image = BeautifulSoup(html_featured_image,'html.parser')

In [91]:
details_link = soup_featured_image.find('article', class_='carousel_item')
details_link_parsed = details_link['style'][22:-1].strip(')').strip("'")
details_link_parsed

'/spaceimages/images/wallpaper/PIA18048-1920x1200.jpg'

In [93]:
domain_link = url_featured_image.split("/")
domain_link

['https:', '', 'www.jpl.nasa.gov', 'spaceimages', '?search=&category=Mars']

In [95]:
featured_image_url = f"{domain_link[0]}//{domain_link[2]}{details_link_parsed}"
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18048-1920x1200.jpg'

### Mars Weather

In [96]:
weather_url = "https://twitter.com/marswxreport?lang=en"

In [97]:
browser.visit(weather_url)

In [116]:
html_weather = browser.html
soup_weather = BeautifulSoup(html_weather,'html.parser')

mars_weathers = soup_weather.find_all('span')
for tag in mars_weathers:
    #print(tag.text)
    if "InSight" in tag.text:
        mars_weather = tag.text

mars_weather

'InSight sol 542 (2020-06-05) low -92.2ºC (-134.0ºF) high 0.0ºC (32.0ºF)\nwinds from the SW at 5.2 m/s (11.6 mph) gusting to 18.3 m/s (41.0 mph)\npressure at 7.30 hPa'

### Mars Facts Table

In [117]:
facts_url = "https://space-facts.com/mars/"

In [118]:
browser.visit(facts_url)

In [130]:
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [131]:
df = tables[0]
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [133]:
df.rename(columns={0:"Attribute",1:"Value"},inplace=True)
df.set_index("Attribute",inplace=True)
df

,Value
Attribute,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [141]:
html_table = df.to_html()
html_table.replace('\n', '') #If it looks weird on the page, change to inplace=True 
df.to_html('table.html')
#!open table.html

In [140]:
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Attribute</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n 

### Mars Hemispere Images

In [138]:
url_hemispheres_start = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"


In [139]:
browser.visit(url_hemispheres_start)


### MongoDB and Flask Application

Need to get everything into a dictionary first

In [160]:
mars_info = {'News':
                {'Headline': article_titles[1].text,
                'Summary': article_summary},
            'FeaturedImage': featured_image_url,
            'Weather': mars_weather,
            'Facts':html_table}  #Add hemisphere images

In [159]:
mars_info

{'News': {'Headline': "NASA's Mars Rover Drivers Need Your Help",
  'Summary': <div class="article_teaser_body">Taken with the infrared camera aboard NASA's Odyssey orbiter, they reveal temperature variations on the small moon as it drifts into and out of Mars’ shadow.</div>},
 'FeaturedImage': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18048-1920x1200.jpg',
 'Weather': 'InSight sol 542 (2020-06-05) low -92.2ºC (-134.0ºF) high 0.0ºC (32.0ºF)\nwinds from the SW at 5.2 m/s (11.6 mph) gusting to 18.3 m/s (41.0 mph)\npressure at 7.30 hPa',
 'Facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Attribute</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 ×